In [1]:
import time

import pandas as pd

In [2]:
# 타이머 기능
def print_time(end, start):
    print('='*50)
    elapsed = end - start
    print('{} secs'.format(round(elapsed)))

In [ ]:
st = time.time()

# 데이터 읽어오기
trn = pd.read_csv('../data/train_ver2.csv')

print_time(time.time(),st)

In [ ]:
st = time.time()

# 변수명 확인
print(trn.columns)

print_time(time.time(),st)

In [ ]:
st = time.time()

# 데이터 첫 10줄 미리보기
print(trn.head())

print_time(time.time(),st)

In [ ]:
st = time.time()

# 데이터 첫 10줄 미리보기
print(trn.info())

print_time(time.time(),st)